## Installing Hugging Face Transformers And Dependencies

In [2]:
#%pip install transformers

In [1]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

## Loading the summarization pipeline

In [2]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to t5-small (https://huggingface.co/t5-small)
2022-03-01 23:15:51.413492: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-01 23:15:51.432849: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Getting blog post from a website

In [3]:
URL = "https://ai.stanford.edu/blog/introduction-to-knowledge-graphs/"

In [4]:
r = requests.get(URL)

In [6]:
r

<Response [200]>

In [7]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.findAll(['h3','p'])

In [8]:
results

[<p class="meta">
 <a href="https://www.linkedin.com/in/vinay-k-chaudhri-849556/">Vinay K. Chaudhri</a>, <a href="https://www.linkedin.com/in/nchittar/">Naren Chittar</a>, <a href="http://logic.stanford.edu/people/genesereth/genesereth.html">Michael Genesereth</a>
 <div class="post-date">May 10, 2021</div>
 </p>,
 <p>Knowledge Graphs (KGs) have emerged as a compelling abstraction for organizing the world’s structured knowledge, and as a way to integrate information extracted from multiple data sources. Knowledge graphs have started to play a central role in representing the information extracted using natural language processing and computer vision. Domain knowledge expressed in KGs is being input into machine learning models to produce better predictions. Our goals in this blog post are to (a) explain the basic terminology, concepts, and usage of KGs, (b) highlight recent applications of KGs that have led to a surge in their popularity, and (c) situate KGs in the overall landscape of 

In [9]:
text = [result.text for result in results]
article = ' '.join(text)

## Data preprocessing

In [12]:
#making the sentances
article = article.replace('.','.<eos>')
article = article.replace('!','!<eos>')
article = article.replace('?','?<eos>')
sentences = article.split('<eos>')

In [14]:
#chuncking the text
max_chunk = 500
current_chunk = 0
chunks=[]
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [15]:
len(chunks[1].split(' '))

481

## Summarizing 

In [16]:
summary = summarizer(chunks,max_length=100,min_length=30,do_sample=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (641 > 512). Running this sequence through the model will result in indexing errors


In [17]:
' '.join([summ['summary_text'] for summ in summary])

'knowledge graphs have emerged as a compelling abstraction for organizing the world’s structured knowledge . KGs are being input into machine learning models to produce better predictions . this blog post is a good starting point before reading a more extensive survey or following research seminars . the use of directed labeled graphs as a data structure has led to a recent surge in the popularity of knowledge graphs . a google search for “Winterthur Zurich” returns the result shown in the left panel of Figure 2 and a relevant portion from Wikipedia in the panel on the right is also known as an Infobox . a fragment of the Wikidata knowledge graph includes data from several independent providers such as the Library of Congress . the KG in Figure 3 shows other objects to which Winterthur and Ontario are connected . a well-documented list of relations in Schema gives us two advantages . a 360-degree view of a customer is created by integrating external data with internal company informati

## Into file

In [ ]:
text = ' '.join([summ['summary_text'] for summ in res])
with open('blogsummary.txt', 'w') as f:
    f.write(text)